# Analisis de prestaciones por diagnostico

- En este apartado sequiere ver la cantidad de prestaciones que se realizan por cada diagnostico principal.

- El resultado que se quiere obtener es parecido al siguiente:

|Año|Diagnostico|Resumen Procedimientos|
|---|-----------|----------------------|
|2019|"I25.1"|{"45.23": 20, "73.59": 10}|
|2020|"I25.1"|{"45.23": 7, "73.59": 3}|

In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)